# Introduction

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import ttest_ind
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split

%matplotlib inline

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        path = os.path.join(dirname, filename)

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv(path)
df.head()

In [ ]:
subjs = ['math score', 'reading score', 'writing score']

In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(16,5))

for i in range(len(subjs)):
  sns.histplot(df[subjs[i]], kde=True, stat="density", linewidth=0, ax=ax[i])

fig.suptitle('Distribution of scores', fontsize=16)
fig.show()

In [ ]:
pd.DataFrame(df[subjs].mean(), columns=['Mean score'])

So the mean scores for the three subjects are similar.

# Student gender factor

Consider how the performance indicators differ by gender.

In [ ]:
df.groupby('gender').agg(count=('gender', 'count'),
                         mean_math=('math score', 'mean'),
                         mean_reading=('reading score', 'mean'),
                         mean_writing=('writing score', 'mean')).reset_index()

In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(16,5))

for i in range(len(subjs)):
  sns.kdeplot(df[df.gender == 'female'][subjs[i]], ax=ax[i], label='female')
  sns.kdeplot(df[df.gender == 'male'][subjs[i]], ax=ax[i], label='male')
  ax[i].legend()

fig.suptitle('Distribution of scores from gender', fontsize=16)
fig.show()

Girls excel at reading and writing, and boys excel at math. See how statistically significant these differences are.

$ H_0 $ is the hypothesis that the mean scores for boys and girls do not really differ. <br>
$ M_f = M_m $ <br>
$ H_1 $ is a hypothesis that the mean grades really differ. <br>
$ M_f \neq M_m $ <br>

Let's calculate the necessary indicators for the criterion of statistical significance. The number of observations is much more than 30, and the graphs show that the data have a normal distribution.

In [ ]:
df_stat = df.groupby('gender').agg(count=('gender', 'count'),
                         mean=('math score', 'mean'),
                         std=('writing score', 'std')).reset_index()
df_stat

In [ ]:
SE = np.sqrt(df_stat.loc[0, 'std']**2/df_stat.loc[0, 'count'] + df_stat.loc[1, 'std']**2/df_stat.loc[1, 'count'])
print('standard error of the mean: {:.2}'.format(SE))

In [ ]:
t_stud = (df_stat.loc[0, 'mean'] - df_stat.loc[1, 'mean'])/SE
print('empirical value of t: {:.2}'.format(t_stud))

In [ ]:
dof = df_stat.loc[1, 'count'] + df_stat.loc[0, 'count'] - 2
print('degrees of freedom: {}'.format(dof))

For the obtained parameters dof and t, the probability of obtaining such or even more significant differences between the two samples is extremely small. We use SciPi to get the p-level of significance.

In [ ]:
t_stud, p = ttest_ind(df[df.gender == 'female']['math score'], df[df.gender == 'male']['math score'])
print('Math\nt={:.2f}, p={:.2e}'.format(t_stud, p))

Likewise with reading and writing.

In [ ]:
t_stud, p = ttest_ind(df[df.gender == 'female']['reading score'], df[df.gender == 'male']['reading score'])
print('Reading\nt={:.2f}, p={:.2e}'.format(t_stud, p))

In [ ]:
t_stud, p = ttest_ind(df[df.gender == 'female']['reading score'], df[df.gender == 'male']['reading score'])
print('Reading\nt={:.2f}, p={:.2e}'.format(t_stud, p))

Thus, we can reject the null hypothesis and regard the mean test scores between boys and girls as statistically significant. Those. boys' mean scores in math are higher than girls', and girls' mean scores are higher in reading and writing.

# Racial or ethnicity

Consider the relationship between the mean score and race or ethnicity.

In [ ]:
df_plot = df.groupby('race/ethnicity')[subjs].mean().plot.bar(title='Mean scores from race/ethnicity')

We see that the best results are shown by students in group E, and their mean score in math is higher than in reading and writing.

# Test preparation courses

Let us now illustrate how the mean score in each of the exams depends on whether the student has taken preparatory courses or not.

In [ ]:
df_test = df.groupby('test preparation course').agg(count=('test preparation course', 'count'),
                         mean_math=('math score', 'mean'),
                         mean_reading=('reading score', 'mean'),
                         mean_writing=('writing score', 'mean')).reset_index()

df_test

In [ ]:
plt.bar(subjs, df_test.iloc[0, 2:], label='completed')
plt.bar(subjs, df_test.iloc[1, 2:], label='none')
plt.legend(title='test preparation course')
plt.show()

The bar charts show that preparatory courses improve student averages.

# The good dinner factor

In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(15, 6))

for i in range(len(subjs)):
  sns.boxplot(data=df, x='lunch', y=subjs[i], ax=ax[i], notch=True)

fig.suptitle('BoxPlot of scores from lunch', fontsize=16)
fig.show()

There is also a relationship between scores and lunch. The mean scores are higher for those students who ate before the exam, and these scores do not fall within the confidence intervals of the mean for those who skipped lunch or ate poorly. Those. we can talk about a statistically significant relationship between academic performance and a good lunch.

# Parent education   

Now let's see how parenting education affects student grades.

In [ ]:
fig, ax = plt.subplots(3, 1, figsize=(15, 12))

for i in range(len(subjs)):
  sns.boxplot(data=df, x='parental level of education', y=subjs[i], notch=True, ax=ax[i])

fig.suptitle('BoxPlot of scores from parental level of education', fontsize=16)
fig.show()

* The averages for high school and some high school are roughly similar, and lower than the rest.
* The highest scores are for students whose parents have a master's degree. At the same time, the minimum score for this group is much higher. Bachelor's degree students are approaching her.
* Associate's degree and some college groups have similar scores. They can be said to show an mean result.

# Predicting

In [ ]:
sns.heatmap(df[subjs].corr(), annot=True, cmap= 'coolwarm').set_title('Correlation of scores')
plt.show()

There is a strong correlation between exam results, so we will not use them together for further analysis. Let's apply and compare a support vector machine and a random forest to predict student grades.

First, we convert categorical features to numeric codes.

In [ ]:
df_num = df

df_num['gender'] = pd.Categorical(df['gender']).codes
df_num['race/ethnicity'] = pd.Categorical(df['race/ethnicity']).codes
df_num['parental level of education'] = pd.Categorical(df['parental level of education']).codes
df_num['lunch'] = pd.Categorical(df['lunch']).codes
df_num['test preparation course'] = pd.Categorical(df['test preparation course']).codes

Let's convert the marks from a 100-point system to a 5-point scale.

In [ ]:
def score_to_mark(score):
  if score < 60:
    return 2
  elif score >= 60 and score < 70:
    return 3
  elif score >= 70 and score < 90:
    return 4
  else:
    return 5

In [ ]:
df_num['math mark'] = df['math score'].apply(score_to_mark)
df_num['reading mark'] = df['reading score'].apply(score_to_mark)
df_num['writing mark'] = df['writing score'].apply(score_to_mark)

The dataset will look like this.

In [ ]:
df_num.head()

In [ ]:
#we divide the data of the training and test samples
X_train, X_test, y_train, y_test = train_test_split(df_num.drop(subjs+['math mark', 'reading mark', 'writing mark'], axis=1), df_num['math mark'])

Using heatmaps to customize models.

In [ ]:
row = []
matrix = []
row_p = []
params = []

for d in range(2, 6, 1):
  row = []
  row_p = []
  for n in range(100, 1000, 50):
    forest = RandomForestClassifier(max_depth=d, n_estimators=n)
    forest.fit(X_train, y_train)
    row.append(forest.score(X_test, y_test))
    row_p.append((d, n))
    
  matrix.append(row)
  params.append(row_p)

In [ ]:
plt.figure(figsize=(10, 4))
sns.heatmap(matrix, annot=True).set_title("HeatMap of model's params")
plt.show()

In [ ]:
params[0][12]

The random forest model performs best with max_depth = 2 and n_estimators = 700

In [ ]:
forest = RandomForestClassifier(max_depth=2, n_estimators=700)
forest.fit(X_train, y_train)

In [ ]:
print(f'Test score: {forest.score(X_test, y_test)}')
print(f'Train score: {forest.score(X_train, y_train)}')

The random forest provides information about the importance of features from the point of view of a given model.

Thus, lunch and race / ethnicity are the most influential factors in making decisions.

In [ ]:
plt.figure(figsize=(8, 3))
plt.barh(y=df_num.drop(subjs+['math mark', 'reading mark', 'writing mark'], axis=1).columns,
        width=forest.feature_importances_)
plt.show()

In [ ]:
row = []
matrix = []
row_p = []
params = []

for g in range(1, 100, 10):
  row = []
  row_p = []
  for c in range(1000, 1, -100):
    svc = SVC(gamma=g/100, C=c/1000)
    svc.fit(X_train, y_train)
    row.append(svc.score(X_test, y_test))
    row_p.append((g/100, c/1000))
    
  matrix.append(row)
  params.append(row_p)

In [ ]:
plt.figure(figsize=(6, 4))
sns.heatmap(matrix, annot=True).set_title("HeatMap of model's params")
plt.show()

In [ ]:
params[1][8]

The best performance was achieved with gamma = 0.11 and C = 0.2

In [ ]:
svc = SVC(gamma=0.11, C=0.2)
svc.fit(X_train, y_train)

In [ ]:
print(f'Test score: {svc.score(X_test, y_test)}')
print(f'Train score: {svc.score(X_train, y_train)}')

Both models showed poor results on test samples and it is impossible to use them to predict the results of the exam in mathematics. But the random forest provided some insight into the importance of the traits.

# Conclusion

Each of the traits in the set has an impact on student performance, but a model that predicts the score has failed.